## Constructing $G$

In [ ]:
# Our first step is to construct $G$ as a NetworkX graph.

# This simplest (albiet, most verbose) way to do this is by using a list of edges
# For example, if you wanted to construct a J4 (a K4 missing an edge)
# you can do it like so:
J4 = nx.Graph( [ (0,1), (0,3), (1,2), (1,3), (2,3) ] )

# To see what this looks like, we've defined a special function above that uses
# NetworkX's drawing capabilities
drawJ4(J4)

In [ ]:
# Now it's your turn! Complete the following definition of G to construct
# the sender described in the introduction. When constructing the graph, use the
# vertex labels 0, 1, 2, 3, 4, 5, and 6, and make sure that (0,1) and (5,6)
# are your sender edges

# Fun fact: instead of listing the edges, you can also construct the graph
# using NetworkX's built-in functions, nx.complete_graph(.) and nx.compose(.)
# but you should do whatever feels easier.

G = nx.Graph([(0,1),(5,6)]) # replace this line

# Once you've constructed the graph, you can run the next cell function to draw
# it and verify whether you've done it correctly. The sender edges will be bolded

In [ ]:
drawG(G)

## Encoding $(K_3, K_3)$-nonarrowing as boolean formulas

Recall that a graph is $(K_3, K_3)$-good coloring of $G$ is one where there are no red triangles and no blue triangles, when the edges of $G$ are colored red and blue. We will now see how to encode this as a boolean formula.

Suppose we represent each edge $e \in E(G)$ as a variable $x_e$. Note that $e$ has two state's w.r.t. what color it can be: red or blue. Similarly, $x_e$ can either be true or false. Thus, we'll say that $x_e$ is true iff $e$ is blue (the rhyme should make this correspondence easy to remember 😊).

Thus, if we can construct or-clause(s) for each triangle that don't allow all of its edges to be the same color, the conjunction of all these clauses must give us a formula $\phi$ such that each satisfying assignment of $\phi$ will correspond to a good coloring of $G$.

Below, we will see how to represent boolean formulas in Z3 and find their satisfying assignments.

### Finding Triangles in Graphs

In [ ]:
# First, we'll need a function that checks whether three given edges
# form a triangle. This is easily done by checking the indices of the edges'
# vertices, or counting the number of vertices and edges on the graph induced
# by the given edges. Or, if you want to be clever, you can use some of
# NetworkX's built-in isomorphism checking functions

# You can assume that e1 e2 and e3 are 2-tuples of integers
def isK3( e1, e2, e3 ):
  return False

In [ ]:
# As a test, check that the following function returns True

isK3( (5,6), (5,231), (6,231) )

In [ ]:
# the following returns False

isK3( (5,6), (5,231), (6,232) )

In [ ]:
# the following returns False

isK3( (5,6), (5,231), (5,4) )

### Constructing $G$'s formula

Complete the code below to construct our desired formula.

In [ ]:
edges = G.edges() # get list of edges

# First, we map each edge to a unique boolean variable

eMap = {} # a dictionary mapping each edge in E(G) to a variable
eMapInv = {} # the inverse of the dictionary above

idx = 1
for e in edges:
    b_name = 'x'+str(idx)
    b = Bool(b_name)
    eMap[e] = b
    eMapInv[b_name] = e
    idx += 1
s = Solver() # initialize solver

for e1, e2, e3 in itertools.combinations( edges, 3 ): # iterate over all 3-tuples of edges
    # if e1 e2 and e3 form a triangle
    if ( isK3(e1, e2, e3) ):
        # add clauses to solver

        e1Var = eMap[e1]
        e2Var = eMap[e2]
        e3Var = eMap[e3]

        # ADD YOUR CODE HERE

        s.add( Or(e1Var) ) # replace this line
        s.add( Or(Not(e2Var)) ) # replace this line

# we have defined a function that, given a Z3 solver, obtains the list of all
# possible solutions
good_colorings = list_all_solutions( s, list(eMap.values()) )

In [ ]:
# Assuming everything is done correctly, we should have 24 solutions,
# each of which corresponds to a good coloring of G
print(len(good_colorings))

In [ ]:
# To view these solutions, run the code in this cell with an index between 0
# and 23 to view one of the good colorings of G
drawGandColoring(G, eMapInv, good_colorings, index=0)

Now, we have two ways to verify that $(0,1)$ and $(5,6)$ are always the same color

1.   Enumerate over all colorings and check that the edges are always the same color
2.   Add extra clause(s) to $\phi$ that forces $(0,1)$ and $(5,6)$ to be the same color and verify that there are no solutions to this new formula

We will opt for the latter, simpler solution, given our newfound SMT-solving prowess.

In [ ]:
# We must first reinitialize our solver, since enumerating over all solutions
# makes the solver unsatisfiable

s = Solver() # initialize solver

for e1, e2, e3 in itertools.combinations( edges, 3 ): # iterate over all 3-tuples of edges
    # if e1 e2 and e3 form a triangle
    if ( isK3(e1, e2, e3) ):
        # add clauses to solver

        e1Var = eMap[e1]
        e2Var = eMap[e2]
        e3Var = eMap[e3]

        # ADD YOUR CODE HERE

        s.add( Or(e1Var) ) # replace this line, same as before
        s.add( Or(Not(e2Var)) ) # replace this line, same as before


# Add the new clause(s) to phi below

e1Var = eMap[ (0,1) ]
e2Var = eMap[ (5,6) ]

# ADD YOUR CODE HERE

s.add( Or(Not(e2Var)) ) # replace this line

# The following should return unsat
s.check()